#Regressione Lineare con Spark MLlib
In questo video vedremo come eseguire una semplice regressione, utilizzando un modello di Regressione Lineare, con il modulo MLlib di Spark. Il modello che andremo a creare avrà lo scopo di stimare il valore di un'abitazione utilizzando un set di proprietà:

**CRIM** Tasso di criminalità per capita<br>
**ZN** Percentuale di terreni residenziali suddivisi in zone per lotti superiori a 25.000 sq.ft.<br>
**INDUS** Percentuale di ettari di attività non al dettaglio per città.<br>
**CHAS** Variabile dummy che indica la prossimità al fiume Charles.<br>
**NOX** Concentrazione di ossido d'azoto (parti per 10 milioni).<br>
**RM** Numero medio di stanze per abitazione<br>
**AGE** Percentuale di abitazione occupate costruite dopo il 1940<br>
**DIS** Media pesata delle distanze da 5 centri lavorativi di Boston.<br>
**RAD** Indice di accessibilità ad autostrade<br>
**TAX** Aliquota dell'imposta sulla proprietà a valore pieno in 10.000 USD.<br>
**PRATIO** Rapporto studente-insegnante per città.<br>
**BLACK** 1000(Bk - 0.63)^2 dove Bk è la percentuale di abitanti di colore per città<br>
**LSTAT** Percentuale della popolazione povera<br>
**MEDV** Mediana del valore di abitazioni occupate in 1.000 USD.<br>

## Procuriamoci il Dataset e importiamolo in un DataFrame
Il dataset che utilizzeremo è il **Boston Housing Dataset**, possiamo ottenerlo [tramite Kaggle](https://www.kaggle.com/c/boston-housing)

### PER DATABRICKS
Se sei su DataBricks devi scaricare il dataset [dalla pagina di Kaggle](https://www.kaggle.com/c/boston-housing/data) e creare la tua tabella. Dopodiché puoi importare i dati in un DataFrame utilizzando una query SQL.

In [4]:
housing_df = spark.sql("SELECT * FROM housing")
housing_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
 ID| crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat|medv|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
 1|0.00632|18.0| 2.31| 0|0.538|6.575|65.2| 4.09| 1|296| 15.3| 396.9| 4.98|24.0|
 2|0.02731| 0.0| 7.07| 0|0.469|6.421|78.9|4.9671| 2|242| 17.8| 396.9| 9.14|21.6|
 4|0.03237| 0.0| 2.18| 0|0.458|6.998|45.8|6.0622| 3|222| 18.7|394.63| 2.94|33.4|
 5|0.06905| 0.0| 2.18| 0|0.458|7.147|54.2|6.0622| 3|222| 18.7| 396.9| 5.33|36.2|
 7|0.08829|12.5| 7.87| 0|0.524|6.012|66.6|5.5605| 5|311| 15.2| 395.6|12.43|22.9|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+
only showing top 5 rows

### PER JUPYTER
Se stai utilizzando Jupyter puoi scaricare il dataset utilizzando le [API di Kaggle per Python](https://github.com/Kaggle/kaggle-api).

In [6]:
!kaggle competitions download boston-housing

Otterremo 3 file CSV: train.csv, test.csv e submission_example.csv. Quello che a noi interessa è il primo, carichiamo all'interno di un DataFrame.

In [8]:
housing_df = spark.read.csv("train.csv", inferSchema=True, header=True)
housing_df.show(5)

##Preprocessing dei dati
Creiamo una lista con i nomi delle colonne che saranno le features del nostro modello, cioè tutte le colonne meno l'ID e il target (MEDV).

In [10]:
features_cols = housing_df.columns[1:-1]

La classe MLlib richiede che le features si trovino tutte all'interno di un unico vettore su di una colonna, possiamo creare questa rappresentazione utilizzando la classe *VectorAssemlber* di MLlib.

In [12]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=features_cols, outputCol="features")
data_df = assembler.transform(housing_df)
data_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
 ID| crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat|medv| features|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
 1|0.00632|18.0| 2.31| 0|0.538|6.575|65.2| 4.09| 1|296| 15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
 2|0.02731| 0.0| 7.07| 0|0.469|6.421|78.9|4.9671| 2|242| 17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
 4|0.03237| 0.0| 2.18| 0|0.458|6.998|45.8|6.0622| 3|222| 18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
 5|0.06905| 0.0| 2.18| 0|0.458|7.147|54.2|6.0622| 3|222| 18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
 7|0.08829|12.5| 7.87| 0|0.524|6.012|66.6|5.5605| 5|311| 15.2| 395.6|12.43|22.9|[0.08829,12.5,7.8...|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+
only showing top 5 rows

E' buona norma portare le features in un range di valori comuni, questo processo può velocizzare anche di molto la fase di addestramento. Facciamolo utilizzando la **normalizzazione** che si esegue sottraendo il valore minimo e poi dividendo per la differenza tra valore massimo e valore minimo. Possiamo eseguire la normalizzazione con MLlib usando la classe *MinMaxScaler*.

In [14]:
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_df)
data_df = scaler_model.transform(data_df)

data_df.show(5)

+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
 ID| crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat|medv| features| scaled_features|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
 1|0.00632|18.0| 2.31| 0|0.538|6.575|65.2| 4.09| 1|296| 15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|[0.0,0.18,0.05814...|
 2|0.02731| 0.0| 7.07| 0|0.469|6.421|78.9|4.9671| 2|242| 17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|[2.85470335157677...|
 4|0.03237| 0.0| 2.18| 0|0.458|6.998|45.8|6.0622| 3|222| 18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|[3.54287862356241...|
 5|0.06905| 0.0| 2.18| 0|0.458|7.147|54.2|6.0622| 3|222| 18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|[8.53146933036738...|
 7|0.08829|12.5| 7.87| 0|0.524|6.012|66.6|5.5605| 5|311| 15.2| 395.6|12.43|22.9|[0.08829,12.5,7.8...|[0.00111481674001...|
+---+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+----+--------------------+--------------------+
only showing top 5 rows

Prossimo passo, dividere il DataFrame con le features preprocessate in due DataFrame, uno per l'addestramento e uno per il testing del modello, possiamo farlo utilizzando il metodo *randomSplit* all'interno della quale dobbiamo passare una lista con la percentuale di osservazioni da assegnare ad ognuno dei DataFrame.<br>
Nel nostro caso assegnamo il 70% degli esempi al set di addestramento e il 30% al set di test.

In [16]:
train_df, test_df = data_df.randomSplit([0.7, 0.3])

print("%d esempi nel train set" % train_df.count())
print("%d esempi nel test set" % test_df.count())

228 esempi nel train set
105 esempi nel test set

Ottimo ! Possiamo creare il modello di Regressione Lineare, usiamo la classe *LinearRegression, all'interno del costruttore dovremo passare due parametri:
* **featuresCol**: il nome della colonna con le features
* **labelCol**: il nome della colonna con il target

In [18]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol="scaled_features", labelCol="medv")

Avviamo l'addestramento con il metodo *fit*, passando al suo interno il set di addetramento

In [20]:
model = lr.fit(train_df)

Abbiamo creato il nostro modello ! Ora verifichiamone la qualità testandolo su dati che non ha visto durante l'addestramento, possiamo farlo usando il test set e il metodo *evalualte*.

In [22]:
evaluation = model.evaluate(test_df)

Il metodo *evaluate* calcolerà diverse metriche che ci possono aiutare a comprendere la qualità del modello, vediamone alcune.

#### MAE - Mean Absolute Error (Errore medio assoluto)

L'errore medio assoluto consiste nella media della somma del valore assoluto degli errori.

$$ MAE = \frac{\sum_{i=1}^n |y_i-\hat{y}_i|}{n} $$

In [25]:
evaluation.meanAbsoluteError

Out[ 10 ]: 3.720966533497767

#### MSE - Mean Squared Error (Errore quadratico assoluto)

L'errore quadratico medio consiste nella media della somma degli errori al quadrato.

$$ MSE =  \frac{\sum_{i=1}^n (y_i-\hat{y}_i)^2}{n}$$

In [27]:
evaluation.meanSquaredError

Out[ 11 ]: 23.929314807759663

#### RMSE - Root Mean Squared Error (Radice dell'errore quadratico medio)

Il RMSE è la radice dell'errore quadratico medio, questa metrica indica mediamente di quanto il nostro modello si è sbagliato.

$$ RMSE =  \sqrt \frac{\sum_{i=1}^n (y_i-\hat{y}_i)^2}{n}$$

In [29]:
evaluation.rootMeanSquaredError

Out[ 48 ]: 4.350187192445518

#### R2 - Coefficient of determination (Coefficiente di Determinazione)

In pratica R2 (pronuciato R Squared) è una versione standardizzata del MSE che torna un punteggio compreso tra 0 e 1 per il train set, mentre per il test set può assumere anche valori negativi. Essendo una funzione ma di scoring, un suo valore maggiore indica una qualità migliore del modello, il suo valore può essere così interpretato:

* R2_score < 0.3 il modello è inutile.
* 0.3 < R2_score < 0.5 il modello è scarso.
* 0.5 < R2_score < 0.7 il modello è discreto.
* 0.7 < R2_score < 0.9 il modello è buono.
* 0.9 < R2_score < 1 il modello è ottimo.
* R2_score = 1 molto probabilmente c'è un errore nel modello.

$$ R^2 = 1-\frac{RSS}{SST} $$

dove RSS è la somma dei quadrati residui:
$$ RSS = \sum_{i=1}^{N}(Y_i-\hat{Y}_i)^2 $$

ed SST è la somma dei quadrati totali:
$$ SST = \sum_{i=1}^{N}(Y_i-\bar{Y})^2 $$

In [33]:
evaluation.r2

Out[ 49 ]: 0.7347789322032157

## Testiamo il Modello
Ora che abbiamo un modello addestrato e funzionante testiamolo su nuovi dati, mettiamo caso che un'agenzia immobiliare ti abbia mandato un file CSV con le proprietà di 10 abitazioni, per la quale tu devi stimare il prezzo usando il modello che hai addestrato. Iniziamo scaricando il file CSV.

In [35]:
!wget https://raw.githubusercontent.com/ProfAI/bigdata/master/6%20-%20Machine%20Learning%20Supervisionato%20-%20Regressione/data/houses.csv

Carichiamolo all'interno di un DataFrame.

In [37]:
houses_df = spark.read.csv("file:/databricks/driver/houses.csv", inferSchema=True, header=True)
houses_df.show(10)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+
 crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+
0.05789|12.5| 6.07| 0|0.409|5.878|21.4| 6.498| 4|345| 18.9|396.21| 8.1|
0.13554|12.5| 6.07| 0|0.409|5.594|36.8| 6.498| 4|345| 18.9| 396.9|13.09|
0.08826| 0.0|10.81| 0|0.413|6.417| 6.6|5.2873| 4|305| 19.2|383.73| 6.72|
0.09164| 0.0|10.81| 0|0.413|6.065| 7.8|5.2873| 4|305| 19.2|390.91| 5.52|
0.19539| 0.0|10.81| 0|0.413|6.245| 6.2|5.2873| 4|305| 19.2|377.17| 7.54|
0.07896| 0.0|12.83| 0|0.437|6.273| 6.0|4.2515| 5|398| 18.7|394.92| 6.78|
0.09512| 0.0|12.83| 0|0.437|6.286|45.0|4.5026| 5|398| 18.7|383.23| 8.94|
0.10153| 0.0|12.83| 0|0.437|6.279|74.5|4.0522| 5|398| 18.7|373.66|11.97|
0.08707| 0.0|12.83| 0|0.437| 6.14|45.8|4.0905| 5|398| 18.7|386.96|10.27|
0.04741| 0.0|11.93| 0|0.573| 6.03|80.8| 2.505| 1|273| 21.0| 396.9| 7.88|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+

Creiamo la colonna con le features.

In [39]:
input_df = assembler.transform(houses_df)
input_df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+
 crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat| features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+
0.05789|12.5| 6.07| 0|0.409|5.878|21.4| 6.498| 4|345| 18.9|396.21| 8.1|[0.05789,12.5,6.0...|
0.13554|12.5| 6.07| 0|0.409|5.594|36.8| 6.498| 4|345| 18.9| 396.9|13.09|[0.13554,12.5,6.0...|
0.08826| 0.0|10.81| 0|0.413|6.417| 6.6|5.2873| 4|305| 19.2|383.73| 6.72|[0.08826,0.0,10.8...|
0.09164| 0.0|10.81| 0|0.413|6.065| 7.8|5.2873| 4|305| 19.2|390.91| 5.52|[0.09164,0.0,10.8...|
0.19539| 0.0|10.81| 0|0.413|6.245| 6.2|5.2873| 4|305| 19.2|377.17| 7.54|[0.19539,0.0,10.8...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+
only showing top 5 rows

Applichiamo la normalizzazione, assicurandoci di applicare la stessa trasformazione che abbiamo applicato agli esempi di addestramento. In che modo ? Utilizzando solamente il meotodo *transform* dello stesso oggetto sulla quale abbiamo già eseguito *fit* sui dati di addestramento.

In [41]:
input_df = scaler_model.transform(input_df)
input_df.show(5)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+
 crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat| features| scaled_features|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+
0.05789|12.5| 6.07| 0|0.409|5.878|21.4| 6.498| 4|345| 18.9|396.21| 8.1|[0.05789,12.5,6.0...|[7.01367564748997...|
0.13554|12.5| 6.07| 0|0.409|5.594|36.8| 6.498| 4|345| 18.9| 396.9|13.09|[0.13554,12.5,6.0...|[0.00175743100090...|
0.08826| 0.0|10.81| 0|0.413|6.417| 6.6|5.2873| 4|305| 19.2|383.73| 6.72|[0.08826,0.0,10.8...|[0.00111440873095...|
0.09164| 0.0|10.81| 0|0.413|6.065| 7.8|5.2873| 4|305| 19.2|390.91| 5.52|[0.09164,0.0,10.8...|[0.00116037775110...|
0.19539| 0.0|10.81| 0|0.413|6.245| 6.2|5.2873| 4|305| 19.2|377.17| 7.54|[0.19539,0.0,10.8...|[0.00257140906470...|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+
only showing top 5 rows

Adesso utilizziamo il meotod *predict* del modello per ottenere la sua predizione, che verrà inserita all'interno di una colonna 'prediction'.

In [43]:
pred_df = model.transform(input_df)
pred_df.show(10)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+
 crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat| features| scaled_features| prediction|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+
0.05789|12.5| 6.07| 0|0.409|5.878|21.4| 6.498| 4|345| 18.9|396.21| 8.1|[0.05789,12.5,6.0...|[7.01367564748997...|21.167081984491425|
0.13554|12.5| 6.07| 0|0.409|5.594|36.8| 6.498| 4|345| 18.9| 396.9|13.09|[0.13554,12.5,6.0...|[0.00175743100090...|16.457652802065986|
0.08826| 0.0|10.81| 0|0.413|6.417| 6.6|5.2873| 4|305| 19.2|383.73| 6.72|[0.08826,0.0,10.8...|[0.00111440873095...|25.903681640078574|
0.09164| 0.0|10.81| 0|0.413|6.065| 7.8|5.2873| 4|305| 19.2|390.91| 5.52|[0.09164,0.0,10.8...|[0.00116037775110...|25.467141594214155|
0.19539| 0.0|10.81| 0|0.413|6.245| 6.2|5.2873| 4|305| 19.2|377.17| 7.54|[0.19539,0.0,10.8...|[0.00257140906470...| 24.58605616870984|
0.07896| 0.0|12.83| 0|0.437|6.273| 6.0|4.2515| 5|398| 18.7|394.92| 6.78|[0.07896,0.0,12.8...|[9.87925924052106...|26.412595765849634|
0.09512| 0.0|12.83| 0|0.437|6.286|45.0|4.5026| 5|398| 18.7|383.23| 8.94|[0.09512,0.0,12.8...|[0.00120770680142...|24.376009451694415|
0.10153| 0.0|12.83| 0|0.437|6.279|74.5|4.0522| 5|398| 18.7|373.66|11.97|[0.10153,0.0,12.8...|[0.00129488473608...|22.748344077041057|
0.08707| 0.0|12.83| 0|0.437| 6.14|45.8|4.0905| 5|398| 18.7|386.96|10.27|[0.08707,0.0,12.8...|[0.00109822437179...|23.521375901623532|
0.04741| 0.0|11.93| 0|0.573| 6.03|80.8| 2.505| 1|273| 21.0| 396.9| 7.88|[0.04741,0.0,11.9...|[5.58836401697426...|23.977299985195426|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+--------------------+--------------------+------------------+

Fantastico ! Ora rimuoviamo le colonne col le features, il prezzo è rappresentato in $10.000, quindi moltiplichiamo per questa cifra per ottenere il prezzo reale e rinominiamo la colonna 'prediction' in 'estimanted_price'.

In [45]:
from pyspark.sql.functions import round

pred_df = pred_df.drop("features") \
          .drop("scaled_features") \
          .withColumn("estimated_price", round(pred_df["prediction"]*10000, 2)) \
          .drop("prediction")

pred_df.show(10)

+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+---------------+
 crim| zn|indus|chas| nox| rm| age| dis|rad|tax|ptratio| black|lstat|estimated_price|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+---------------+
0.05789|12.5| 6.07| 0|0.409|5.878|21.4| 6.498| 4|345| 18.9|396.21| 8.1| 211670.82|
0.13554|12.5| 6.07| 0|0.409|5.594|36.8| 6.498| 4|345| 18.9| 396.9|13.09| 164576.53|
0.08826| 0.0|10.81| 0|0.413|6.417| 6.6|5.2873| 4|305| 19.2|383.73| 6.72| 259036.82|
0.09164| 0.0|10.81| 0|0.413|6.065| 7.8|5.2873| 4|305| 19.2|390.91| 5.52| 254671.42|
0.19539| 0.0|10.81| 0|0.413|6.245| 6.2|5.2873| 4|305| 19.2|377.17| 7.54| 245860.56|
0.07896| 0.0|12.83| 0|0.437|6.273| 6.0|4.2515| 5|398| 18.7|394.92| 6.78| 264125.96|
0.09512| 0.0|12.83| 0|0.437|6.286|45.0|4.5026| 5|398| 18.7|383.23| 8.94| 243760.09|
0.10153| 0.0|12.83| 0|0.437|6.279|74.5|4.0522| 5|398| 18.7|373.66|11.97| 227483.44|
0.08707| 0.0|12.83| 0|0.437| 6.14|45.8|4.0905| 5|398| 18.7|386.96|10.27| 235213.76|
0.04741| 0.0|11.93| 0|0.573| 6.03|80.8| 2.505| 1|273| 21.0| 396.9| 7.88| 239773.0|
+-------+----+-----+----+-----+-----+----+------+---+---+-------+------+-----+---------------+

Ora puoi anche salvare il tuo file CSV e mandarlo indietro all'agenzia immobialiare con la stima dei prezzi :).